In [160]:
import pandas as pd
import numpy as np
from datetime import datetime
import shap
import xgboost as xgb
import lightgbm as lgm

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings("ignore")

In [161]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from operator import itemgetter

In [162]:
train = pd.read_csv('train_BRCpofr.csv')
test = pd.read_csv('test_koRSKBP.csv')

In [163]:
income = train['income']
train['income_int'] = np.where(income=='5L-10L', 75000, 
                              np.where(income=='<=2L', 15000, 
                              np.where(income=='More than 10L', 120000, 0)))

income = test['income']
test['income_int'] = np.where(income=='5L-10L', 75000, 
                              np.where(income=='<=2L', 15000, 
                              np.where(income=='More than 10L', 120000, 0)))

In [164]:
num_policies_int_ = train['num_policies']
train['num_policies_int'] = np.where(num_policies_int_=='More than 1', 2,
                                 np.where(num_policies_int_=='1', 1, 0))

num_policies_int_ = test['num_policies']
test['num_policies_int'] = np.where(num_policies_int_=='More than 1', 2,
                                 np.where(num_policies_int_=='1', 1, 0))

In [165]:
train['Num_claims'] = train['claim_amount'].apply(lambda x: 1 if x > 0 else 0)
train['Num_claims'] = train['Num_claims'].astype('int')
train['Claim_amount_per_policy'] = train['claim_amount'] / train['num_policies_int']
train['Claim_amount_per_policy'] = train['Claim_amount_per_policy'].astype('float')
train['Claims_ratio'] = train['Num_claims'] / train['num_policies_int']
train['Claims_ratio'] = train['Claims_ratio'].astype('float')

test['Num_claims'] = test['claim_amount'].apply(lambda x: 1 if x > 0 else 0)
test['Num_claims'] = test['Num_claims'].astype('int')
test['Claim_amount_per_policy'] = test['claim_amount'] / test['num_policies_int']
test['Claim_amount_per_policy'] = test['Claim_amount_per_policy'].astype('float')
test['Claims_ratio'] = test['Num_claims'] / test['num_policies_int']
test['Claims_ratio'] = test['Claims_ratio'].astype('float')

In [166]:
income = train['income']
area = train['area']
train['spending_power'] =  np.where((area=='Urban') & (income=='5L - 10L'), 150000,
                        np.where((area=='Urban') & (income=='2L - 5L'), 350000,
                        np.where((area=='Urban') & (income=='<=2L'), 200000, 
                        np.where((area=='Urban') & (income=='More than 10L'), 1000000,
                        np.where((area=='Rural') & (income=='5L - 10L'), 100000,
                        np.where((area=='Rural') & (income=='2L - 5L'), 350000,
                        np.where((area=='Rural') & (income=='<=2L'), 200000, 
                        np.where((area=='Rural') & (income=='More than 10L'), 1000000, 0))))))))

In [167]:
income = test['income']
area = test['area']
test['spending_power'] =  np.where((area=='Urban') & (income=='5L - 10L'), 150000,
                        np.where((area=='Urban') & (income=='2L - 5L'), 350000,
                        np.where((area=='Urban') & (income=='<=2L'), 200000, 
                        np.where((area=='Urban') & (income=='More than 10L'), 1000000,
                        np.where((area=='Rural') & (income=='5L - 10L'), 100000,
                        np.where((area=='Rural') & (income=='2L - 5L'), 350000,
                        np.where((area=='Rural') & (income=='<=2L'), 200000, 
                        np.where((area=='Rural') & (income=='More than 10L'), 1000000, 0))))))))

In [168]:
#train.drop('num_policies_int', axis=1, inplace=True)
#test.drop('num_policies_int', axis=1, inplace=True)

In [169]:
cat_col = [col for col in train.columns if train[col].dtypes == 'object']
cat_col

['gender',
 'area',
 'qualification',
 'income',
 'num_policies',
 'policy',
 'type_of_policy']

In [120]:
'''def target_encoding(df,cols,target):
    for col in cols:
        means = df.groupby(col)[target].mean()
        df[col] = df[col].map(means)
    return df

# example usage
cols_to_encode = cat_col
target_col = 'cltv'
train = target_encoding(train, cols_to_encode,target_col)'''

"def target_encoding(df,cols,target):\n    for col in cols:\n        means = df.groupby(col)[target].mean()\n        df[col] = df[col].map(means)\n    return df\n\n# example usage\ncols_to_encode = cat_col\ntarget_col = 'cltv'\ntrain = target_encoding(train, cols_to_encode,target_col)"

In [170]:
for col in cat_col:
    le = LabelEncoder()
    train.loc[:, col] = le.fit_transform(train[col])

In [122]:
'''train = pd.get_dummies(data=train, columns=cat_col)'''

'train = pd.get_dummies(data=train, columns=cat_col)'

In [123]:
X = train.drop('cltv', axis=1)
y = train['cltv']
X_constant = sm.add_constant(X)
lin_reg = sm.OLS(y, X_constant).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cltv   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     1010.
Date:                Sun, 22 Jan 2023   Prob (F-statistic):               0.00
Time:                        21:20:20   Log-Likelihood:            -1.1398e+06
No. Observations:               89392   AIC:                         2.280e+06
Df Residuals:                   89375   BIC:                         2.280e+06
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    2.068e+04   1303.920     15.857      0.000    1.81e+04    2.32e+04
id                         -0.0116      0.011     -1.075      0.283      -0.033       0.010
gender                   1947.0306    567.805      3.429      0.001     834.139    3059.923
area                     9258.1844    715.901     12.932      0.000    7855.025    1.07e+04
qualification           -1707.4975    492.125     -3.470      0.001   -2672.058    -742.937
income                  -1579.7648   1328.627     -1.189      0.234   -4183.861    1024.332
marital_status          -6607.0136    571.733    -11.556      0.000   -7727.604   -5486.423
vintage                    56.4627    122.127      0.462      0.644    -182.905     295.830
claim_amount                7.5959      0.361     21.022      0.000       6.888       8.304
num_policies             1.444e+04   1184.160     12.198      0.000    1.21e+04    1.68e+04
policy                    567.2810    433.097      1.310      0.190    -281.584    1416.146
type_of_policy          -1836.3356    411.134     -4.467      0.000   -2642.154   -1030.517
income_int                 -0.0298      0.018     -1.641      0.101      -0.065       0.006
num_policies_int         3.512e+04    500.016     70.237      0.000    3.41e+04    3.61e+04
Num_claims               -737.0516   2892.291     -0.255      0.799   -6405.915    4931.812
Claim_amount_per_policy    -7.3947      0.569    -13.005      0.000      -8.509      -6.280
Claims_ratio            -1644.8822   4081.665     -0.403      0.687   -9644.907    6355.143
spending_power              0.0019      0.002      0.828      0.408      -0.003       0.006
==============================================================================
Omnibus:                    51170.490   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           405369.158
Skew:                           2.729   Prob(JB):                         0.00
Kurtosis:                      11.891   Cond. No.                     2.00e+20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.48e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [124]:
X = train.drop('cltv', axis=1)
y = train['cltv']

pd.set_option('display.max_rows', 67)
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif}, index=X.columns)

,vif
id,1.000247
gender,1.018482
area,1.386815
qualification,1.007215
income,19.529753
marital_status,1.026374
vintage,1.005657
claim_amount,17.859160
num_policies,51.596136
policy,1.035863


In [129]:
X.drop('num_policies_int', axis=1, inplace=True)

In [92]:
X.drop('Num_claims', axis=1, inplace=True)

In [131]:
X.drop('income', axis=1, inplace=True)

In [126]:
X.drop('Claim_amount_per_policy', axis=1, inplace=True)

In [125]:
X.drop('Claims_ratio', axis=1, inplace=True)

In [134]:
X.drop('spending_power', axis=1, inplace=True)

In [137]:
X.drop('policy', axis=1, inplace=True)

In [138]:
X.drop('vintage', axis=1, inplace=True)

In [139]:
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
pd.DataFrame({'vif': vif}, index=X.columns)

,vif
id,3.453147
gender,2.225382
area,4.341514
qualification,1.995012
marital_status,2.225680
claim_amount,5.319287
num_policies,2.863769
type_of_policy,2.738405
income_int,2.952032
Num_claims,8.795471


In [141]:
X_constant = sm.add_constant(X)
lin_reg = sm.OLS(y, X_constant, L1_wt=0.01, L2_wt=0.01).fit()
lin_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   cltv   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     1573.
Date:                Sun, 22 Jan 2023   Prob (F-statistic):               0.00
Time:                        21:24:38   Log-Likelihood:            -1.1399e+06
No. Observations:               89392   AIC:                         2.280e+06
Df Residuals:                   89381   BIC:                         2.280e+06
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           4.788e+04   1298.821     36.862      0.000    4.53e+04    5.04e+04
id                -0.0108      0.011     -0.997      0.319      -0.032       0.010
gender          2304.8179    568.025      4.058      0.000    1191.495    3418.141
area            8470.7271    713.823     11.867      0.000    7071.640    9869.814
qualification  -1609.0631    492.876     -3.265      0.001   -2575.095    -643.031
marital_status -6663.8403    572.255    -11.645      0.000   -7785.454   -5542.226
claim_amount       3.2303      0.119     27.253      0.000       2.998       3.463
num_policies    6.483e+04    608.846    106.475      0.000    6.36e+04     6.6e+04
type_of_policy -1760.6566    411.048     -4.283      0.000   -2566.306    -955.007
income_int        -0.0524      0.007     -7.135      0.000      -0.067      -0.038
Num_claims     -5679.1228    963.142     -5.896      0.000   -7566.872   -3791.373
==============================================================================
Omnibus:                    51270.179   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           405224.373
Skew:                           2.737   Prob(JB):                         0.00
Kurtosis:                      11.878   Cond. No.                     4.25e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.25e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [173]:
train

,id,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv,income_int,num_policies_int,Num_claims,Claim_amount_per_policy,Claims_ratio,spending_power
0,1,1,1,0,1,1,5,5790,1,0,1,64308,75000,2,1,2895.0,0.5,0
1,2,1,0,1,1,0,8,5080,1,0,1,515400,75000,2,1,2540.0,0.5,0
2,3,1,1,0,1,1,8,2599,1,0,1,64212,75000,2,1,1299.5,0.5,0
3,4,0,0,1,1,0,7,0,1,0,1,97920,75000,2,0,0.0,0.0,0
4,5,1,1,1,3,1,6,3508,1,0,0,59736,120000,2,1,1754.0,0.5,1000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89387,89388,1,0,1,3,0,6,0,1,1,1,95736,120000,2,0,0.0,0.0,1000000
89388,89389,1,1,0,1,1,0,3897,1,0,1,67596,75000,2,1,1948.5,0.5,0
89389,89390,0,1,0,1,0,4,3363,1,1,1,114120,75000,2,1,1681.5,0.5,0
89390,89391,0,0,1,0,0,8,0,1,1,1,71772,0,2,0,0.0,0.0,0


In [171]:
X = train[['id', 'gender', 'area', 'income', 'marital_status', 'claim_amount', 'num_policies', 'policy',
                 'type_of_policy', 'education_level', 'avg_claim_per_id', 'no_of_policies_to_vintage_ratio',
                 'income_vintage', 'Frequency_claim_amount', 'Ratio_vintage_to_policy']]
y = train['cltv']

KeyError: "['education_level', 'avg_claim_per_id', 'no_of_policies_to_vintage_ratio', 'income_vintage', 'Frequency_claim_amount', 'Ratio_vintage_to_policy'] not in index"

In [144]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15098093425688142


In [154]:
las_reg = Lasso()
las_reg.fit(X_train, y_train)
y_pred = las_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15098230035524463


In [155]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(X_train, y_train)
y_pred = gb_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15706652217509853


In [156]:
xgb_reg = xgb.XGBRegressor(eta=0.1)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15416849605726635


In [157]:
lgm_reg = lgm.LGBMRegressor()
lgm_reg.fit(X_train, y_train)
y_pred = lgm_reg.predict(X_test)
score = r2_score(y_test, y_pred)
print(score)

0.15358863002469425


In [158]:
perm = PermutationImportance(lin_reg, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names=X.columns.tolist())

Weight,Feature
0.2226 ± 0.0051,num_policies
0.0273 ± 0.0013,claim_amount
0.0041 ± 0.0006,area
0.0031 ± 0.0005,marital_status
0.0012 ± 0.0004,Num_claims
0.0012 ± 0.0003,income_int
0.0005 ± 0.0002,type_of_policy
0.0003 ± 0.0003,gender
0.0003 ± 0.0002,qualification
-0.0000 ± 0.0000,id
